In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

# Data Cleaning & Missing Values

### Load the Data

In [ ]:
#df = pd.read_csv('berlin_housing_with_scraped.csv')
df = pd.read_csv('https://raw.githubusercontent.com/ReDI-School/data-analytics/master/Class%20material/Class-11_Data_cleaning/berlin_housing_with_scraped.csv')

### Check the Data's `shape`,  `head`, and the `dtypes`

In [ ]:
df.shape

(764, 22)

In [ ]:
df.head(3)

,url,region,condition,rooms,rent,year_construction,space,cold_rent,utilities,heating_costs,security_deposit,parking_space,renovated_date,property_condition,furnishing,heating_type,energy_sources,energy_certificate,type_of_certificate,energy_requirement,efficiency_class,tags
0,https://www.immobilienscout24.de/expose/116051687,Mitte,first_time_use,4.0,2659.0,2019,117.20,2.199€,+460€,in,3,120€,NaN,New,NaN,Underfloor Heating,NaN,NaN,Bedarfsausweis,NaN,B,"['Balkon', 'Terrasse Balkon', 'Terrasse Person..."
1,https://www.immobilienscout24.de/expose/115338103,Kreuzberg,first_time_use,1.0,1200.0,2020,29.33,"973,51€","+226,49€",in,"2920,53",NaN,NaN,New,Gehobene Qualität,Central Heating,Fernwärme Energieausweis liegt vor,liegt vor,NaN,NaN,NaN,"['Balkon', 'Terrasse Balkon', 'Terrasse Person..."
2,https://www.immobilienscout24.de/expose/116458710,Köpenick,well_kept,2.0,979.0,1997,83.61,799€,+90€,+90€,Drei,NaN,NaN,Groomed,Normale Qualität,Central Heating,Fernwärme Energieausweis liegt vor,liegt vor,Verbrauchsausweis Energie­ver­brauchs­kennwert...,NaN,NaN,"['Balkon', 'Terrasse Balkon', 'Terrasse Person..."


In [ ]:
df.dtypes

url                     object
region                  object
condition               object
rooms                  float64
rent                   float64
year_construction        int64
space                  float64
cold_rent               object
utilities               object
heating_costs           object
security_deposit        object
parking_space           object
renovated_date         float64
property_condition      object
furnishing              object
heating_type            object
energy_sources          object
energy_certificate      object
type_of_certificate     object
energy_requirement      object
efficiency_class        object
tags                    object
dtype: object

### First Question: Is there missing data? If yes, how much?

In [ ]:
(df.isnull().sum() / len(df)).sort_values(ascending = False)

parking_space          0.882199
renovated_date         0.744764
efficiency_class       0.671466
energy_requirement     0.573298
security_deposit       0.496073
heating_costs          0.442408
cold_rent              0.442408
utilities              0.442408
furnishing             0.417539
energy_certificate     0.409686
type_of_certificate    0.388743
energy_sources         0.353403
property_condition     0.344241
heating_type           0.325916
rooms                  0.000000
region                 0.000000
condition              0.000000
tags                   0.000000
rent                   0.000000
year_construction      0.000000
space                  0.000000
url                    0.000000
dtype: float64

## Clean `tags`

Have a look at the tags columns. What does this information tell us?

In [ ]:
df.tags[0]

"['Balkon', 'Terrasse Balkon', 'Terrasse Personenaufzug Personenaufzug Einbauküche Einbauküche']"

In [ ]:
df.tags.unique()

array(["['Balkon', 'Terrasse Balkon', 'Terrasse Personenaufzug Personenaufzug Einbauküche Einbauküche']"],
      dtype=object)

Because this doesn't give us any information, let's get rid of redundant data.

In [ ]:
df.drop('tags', axis=1, inplace=True)

In [ ]:
df.shape

(764, 21)

## Clean `security_deposit`

In [ ]:
df.security_deposit.value_counts()

3           123
Drei         13
Kaution:      7
3-fache       5
1600          3
           ... 
1.554,00      1
9025,50       1
3905          1
10500         1
4552          1
Name: security_deposit, Length: 216, dtype: int64

Some values clearly are an amount/ For example:

In [ ]:
df.security_deposit[df.security_deposit.str.contains("€").fillna(False)]

36        1.500€
167       1.500€
271    5.394,00€
289        6600€
393       3.597€
403       3.750€
405        5097€
423    2.286,45€
462      6570.0€
464      6675.0€
466      9333.0€
469      9792.0€
490     1074,60€
507    4.698,00€
545    4.197,00€
697        €3500
Name: security_deposit, dtype: object

Other values are impossible to interpret.

In [ ]:
df.security_deposit[df.security_deposit.str.contains("Kaution").fillna(False)]

420    Kaution:
425    Kaution:
429    Kaution:
474    Kaution:
475    Kaution:
495     Kaution
514    Kaution:
515    Kaution:
Name: security_deposit, dtype: object

Which values are numeric?

**First, let's remove the currency**

In [ ]:
type(str(np.nan))

str

In [ ]:
def remove_strings(value, strings_to_remove):
    """
    Remove strings in <strings_to_remove> from <value>. 
    Removes Euro/euro strings by default. 
    Assumes that each string occurs only once.
    """
    
    # return NaN if value is missing
    if pd.isna(value): return np.nan
    
    # cast to string so that string methods work
    value = str(value)
    
    # remove different strings
    for string in strings_to_remove:
        value = value.lower().replace(string, '').strip()
    
    return value

In [ ]:
df['security_deposit'] = df.security_deposit.apply(remove_strings, args=(['€', 'Euro'],)).astype(str)

In [ ]:
df.security_deposit.unique()

array(['3', '2920,53', 'drei', 'nan', '2400', '3507.42', '2000',
       '2139.78', '5.397,00', '3885,00', '7.950,00', '4152', '4.650,00',
       '1.500', '3x', '9.150,-', '4.545', '2xnkm', '6.900,00', '5.994,00',
       '6.000,-', '7.470,00', '5.172,00', '2.145,00', '6.215,70',
       '11.475,00', '14.850,00', '4.197,00', '2600,00', '12.900,00', '2',
       '3450', '5000', '10.000', '3620', '1690', '1790', 'ja', '2990',
       '1490', '9.960,00', '3159,00', '4.350,00', '5010', '4333,90',
       '3000', '2.013,00', '2841', '4000', '2850.72', '3300.75', '2344',
       '3645,00', '2397', '4500', '9.718,00', '3.597,00', '3900',
       '3.780,00', '4800', '3.582', '3630', '2549.79', '3831', '2.274,00',
       '2850', '2960', '4.680', '4350', '3500', '4080', '4050', '2338',
       '5940,00', '8100', '3.859,17', '2.630,43', '6552', '2.700,00',
       '1797,00', '3345', '3360', '5148', '2472', '12.000,00', '6.300,00',
       '5.394,00', '1.554,00', '6600', '1.727,76', '9025,50', '3.750,00',
  

**Now, we need to convert the decimal into something a computer can better read**

In [ ]:
def convert_string_to_float(value):
    """Replace a period with empty stringm and comma with period (decimal point)."""
    return value.replace(".", "").replace(",", ".")


def convert_decimal(value):
    """Convert numeric string to a decimal number."""
    
    # return NaN if missing
    if pd.isna(value):
        return np.nan       
    
    # check if wrong format w/o decimal i.e. 3.500
    if len(value.split('.')[-1]) == 3:
        return convert_string_to_float(value)
    
    # convert to float
    try:                               
        return float(value)
    except:
        return convert_string_to_float(value)

In [ ]:
df['security_deposit'] = df.security_deposit.apply(convert_decimal)

In [ ]:
df.security_deposit.unique()

array([3.0, '2920.53', 'drei', 'nan', 2400.0, 3507.42, 2000.0, 2139.78,
       '5397.00', '3885.00', '7950.00', 4152.0, '4650.00', '1500', '3x',
       '9150.-', '4545', '2xnkm', '6900.00', '5994.00', '6000.-',
       '7470.00', '5172.00', '2145.00', '6215.70', '11475.00', '14850.00',
       '4197.00', '2600.00', '12900.00', 2.0, 3450.0, 5000.0, '10000',
       3620.0, 1690.0, 1790.0, 'ja', 2990.0, 1490.0, '9960.00', '3159.00',
       '4350.00', 5010.0, '4333.90', 3000.0, '2013.00', 2841.0, 4000.0,
       2850.72, 3300.75, 2344.0, '3645.00', 2397.0, 4500.0, '9718.00',
       '3597.00', 3900.0, '3780.00', 4800.0, '3582', 3630.0, 2549.79,
       3831.0, '2274.00', 2850.0, 2960.0, '4680', 4350.0, 3500.0, 4080.0,
       4050.0, 2338.0, '5940.00', 8100.0, '3859.17', '2630.43', 6552.0,
       '2700.00', '1797.00', 3345.0, 3360.0, 5148.0, 2472.0, '12000.00',
       '6300.00', '5394.00', '1554.00', 6600.0, '1727.76', '9025.50',
       '3750.00', 7575.0, '7650.00', '5355.00', 5850.0, '4170.00',

In [ ]:
df.rent.isnull().sum()

0

**And now, let's get a list of non-numeric values**

In [ ]:
def is_float(value):
    try:
        float(value)
        return True
    except:
        return False

df.security_deposit[df.security_deposit.apply(lambda x: not is_float(x))].unique()

array(['drei', '3x', '9150.-', '2xnkm', '6000.-', 'ja', '3-fache',
       'kaution:', 'kaution', '3nkm'], dtype=object)

**Build a map of what the values should be. This is manually set because it's a small amount of data.**

In [ ]:
str_security_deposit_map = {
    'drei': '3',
    '3x': '3',
    '9150.-': 9150,
    '2xnkm': '2',
    '6000.-': 6000,
    '3-fache': '3',
    '3nkm': '3',    
}

str_security_deposit_unclear = ['ja', 'kaution', 'kaution:']

In [ ]:
df['security_deposit'] = df.security_deposit.apply(lambda x: str_security_deposit_map.get(x, x))

In [ ]:
df.security_deposit.unique()

array([3.0, '2920.53', '3', 'nan', 2400.0, 3507.42, 2000.0, 2139.78,
       '5397.00', '3885.00', '7950.00', 4152.0, '4650.00', '1500', 9150,
       '4545', '2', '6900.00', '5994.00', 6000, '7470.00', '5172.00',
       '2145.00', '6215.70', '11475.00', '14850.00', '4197.00', '2600.00',
       '12900.00', 2.0, 3450.0, 5000.0, '10000', 3620.0, 1690.0, 1790.0,
       'ja', 2990.0, 1490.0, '9960.00', '3159.00', '4350.00', 5010.0,
       '4333.90', 3000.0, '2013.00', 2841.0, 4000.0, 2850.72, 3300.75,
       2344.0, '3645.00', 2397.0, 4500.0, '9718.00', '3597.00', 3900.0,
       '3780.00', 4800.0, '3582', 3630.0, 2549.79, 3831.0, '2274.00',
       2850.0, 2960.0, '4680', 4350.0, 3500.0, 4080.0, 4050.0, 2338.0,
       '5940.00', 8100.0, '3859.17', '2630.43', 6552.0, '2700.00',
       '1797.00', 3345.0, 3360.0, 5148.0, 2472.0, '12000.00', '6300.00',
       '5394.00', '1554.00', 6600.0, '1727.76', '9025.50', '3750.00',
       7575.0, '7650.00', '5355.00', 5850.0, '4170.00', '2340.00', 5910.0,
 

In [ ]:
def convert_to_float(value):
    if is_float(value):
        return float(value)
    else:
        return np.nan

# Not storing in df, just want to see output
df.security_deposit[df.security_deposit.apply(lambda x: convert_to_float(x)) <= 3].value_counts()

3.0    123
3       21
2        2
2.0      2
2.5      1
Name: security_deposit, dtype: int64

Let's assume that the unclear values are three months, as it is the most common option.

In [ ]:
unclear_security_deposit_map = {str_security_deposit: '3' for str_security_deposit in str_security_deposit_unclear}

In [ ]:
df['security_deposit'] = df.security_deposit.apply(lambda x: unclear_security_deposit_map.get(x, x))

In [ ]:
df.security_deposit.unique()

array([3.0, '2920.53', '3', 'nan', 2400.0, 3507.42, 2000.0, 2139.78,
       '5397.00', '3885.00', '7950.00', 4152.0, '4650.00', '1500', 9150,
       '4545', '2', '6900.00', '5994.00', 6000, '7470.00', '5172.00',
       '2145.00', '6215.70', '11475.00', '14850.00', '4197.00', '2600.00',
       '12900.00', 2.0, 3450.0, 5000.0, '10000', 3620.0, 1690.0, 1790.0,
       2990.0, 1490.0, '9960.00', '3159.00', '4350.00', 5010.0, '4333.90',
       3000.0, '2013.00', 2841.0, 4000.0, 2850.72, 3300.75, 2344.0,
       '3645.00', 2397.0, 4500.0, '9718.00', '3597.00', 3900.0, '3780.00',
       4800.0, '3582', 3630.0, 2549.79, 3831.0, '2274.00', 2850.0, 2960.0,
       '4680', 4350.0, 3500.0, 4080.0, 4050.0, 2338.0, '5940.00', 8100.0,
       '3859.17', '2630.43', 6552.0, '2700.00', '1797.00', 3345.0, 3360.0,
       5148.0, 2472.0, '12000.00', '6300.00', '5394.00', '1554.00',
       6600.0, '1727.76', '9025.50', '3750.00', 7575.0, '7650.00',
       '5355.00', 5850.0, '4170.00', '2340.00', 5910.0, 6330.0,

The values 1, 2 and 3 refer to the number of months rent as deposit, so let's calculate those values by multiplying with the rent

In [ ]:
df.rent.dtype

dtype('float64')

**Now, convert the month duration columns (3/2/etc Months) to an actual value, based on the rent**

In [ ]:
def convert_months_to_total(row):
    fl_dep = convert_to_float(row.security_deposit)
    if fl_dep <= 3:
        return float(row.rent * fl_dep)
    else:
        return fl_dep

In [ ]:
df = df.assign(deposit_cleaned=df.apply(convert_months_to_total, axis = 1))

df[["security_deposit", "deposit_cleaned", "rent"]].head(50)

,security_deposit,deposit_cleaned,rent
0,3,7977.00,2659.00
1,2920.53,2920.53,1200.00
2,3,2937.00,979.00
3,nan,NaN,1830.22
4,nan,NaN,2272.00
5,3,2520.00,840.00
6,nan,NaN,1509.45
7,3,2192.19,730.73
8,3,1685.79,561.93
9,3,5367.00,1789.00


#### Check Distribution and Descriptive Stats

In [ ]:
px.histogram(df, x="deposit_cleaned", marginal='box')

In [ ]:
df.deposit_cleaned.agg(['mean', 'median', 'std', 'min', 'max'])

mean       4670.031169
median     3900.000000
std        3153.089709
min         842.220000
max       29322.000000
Name: deposit_cleaned, dtype: float64

#### Filling the missing values intuitively

(Note: these are old advertisements and no longer available)

 - https://www.immobilienscout24.de/expose/116573177: has all of it's data mising, *will assume three months*
 - https://www.immobilienscout24.de/expose/115925878: has only some of its data missing, *will assume scraping error and therefore three months*
 - https://www.immobilienscout24.de/expose/114437800: has only deposit missing, *will assume there is no deposit*
 
**Because the data is missing for different reasons, we should use different fill strategies**

In [ ]:
df[df.deposit_cleaned.isna()].head()

,url,region,condition,rooms,rent,year_construction,space,cold_rent,utilities,heating_costs,security_deposit,parking_space,renovated_date,property_condition,furnishing,heating_type,energy_sources,energy_certificate,type_of_certificate,energy_requirement,efficiency_class,deposit_cleaned
3,https://www.immobilienscout24.de/expose/116573177,Wilmersdorf,well_kept,4.0,1830.22,1900,171.18,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.immobilienscout24.de/expose/115925878,Kreuzberg,first_time_use,2.0,2272.00,2020,88.27,NaN,NaN,NaN,nan,NaN,NaN,New,Gehobene Qualität,Central Heating,Fernwärme Energieausweis liegt zur Besichtigun...,liegt zur Besichtigung vor,NaN,NaN,NaN,NaN
6,https://www.immobilienscout24.de/expose/108376992,Mitte,mint_condition,2.0,1509.45,2015,61.61,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,https://www.immobilienscout24.de/expose/114437800,Mitte,first_time_use,4.0,2499.00,2019,125.20,1.999€,+500€,in,nan,120€,NaN,New,NaN,Underfloor Heating,Fernwärme,NaN,Bedarfsausweis,NaN,B,NaN
11,https://www.immobilienscout24.de/expose/116205341,Kreuzberg,first_time_use,2.0,2328.00,2020,90.36,NaN,NaN,NaN,nan,NaN,NaN,New,Gehobene Qualität,Central Heating,Fernwärme Energieausweis liegt zur Besichtigun...,liegt zur Besichtigung vor,NaN,NaN,NaN,NaN


In [ ]:
df['n_missing'] = df[df.columns[-14:]].isnull().sum(axis=1)

In [ ]:
df[df.n_missing == 1].head(3)

,url,region,condition,rooms,rent,year_construction,space,cold_rent,utilities,heating_costs,security_deposit,parking_space,renovated_date,property_condition,furnishing,heating_type,energy_sources,energy_certificate,type_of_certificate,energy_requirement,efficiency_class,deposit_cleaned,n_missing
14,https://www.immobilienscout24.de/expose/116361020,Tiergarten,first_time_use,2.0,1399.65,2019,66.65,"1.199,70€","+133,30€","+66,65€",2400,NaN,2019.0,New,Gehobene Qualität,District Heating,Fernwärme Energieausweis liegt vor,liegt vor,Bedarfsausweis,"66,2 kWh/(m²*a)",B,2400.0,1
29,https://www.immobilienscout24.de/expose/116326551,Charlottenburg,mint_condition,2.0,1810.00,2016,100.47,1.550€,+250€,in,4650.00,150€,NaN,Mint Condition,Luxus,Central Heating,Fernwärme Energieausweis liegt vor,liegt vor,Bedarfsausweis,58 kWh/(m²*a),B,4650.0,1
43,https://www.immobilienscout24.de/expose/116436290,Mitte,no_information,4.0,3677.70,2016,125.54,3.050€,"+426,84€","+200,86€",9150,200€,2016.0,NaN,Luxus,District Heating,Fernwärme Energieausweis liegt vor,liegt vor,Bedarfsausweis,74 kWh/(m²*a),B,9150.0,1


In [ ]:
def fill_missing_security_deposit(row):
  
    # if the value isn't missing, return the value
    if not pd.isna(row.deposit_cleaned):
        return row.deposit_cleaned
  
    # if there is only one missing value, return 0. Otherwise return 3 months of rent
    if row.n_missing == 1:
        return 0
    else:
        return row.rent * 3

In [ ]:
df['deposit_cleaned'] = df.apply(fill_missing_security_deposit, axis=1)

In [ ]:
px.histogram(df, x="deposit_cleaned", marginal='box')

In [ ]:
df.deposit_cleaned.agg(['mean', 'median', 'std', 'min', 'max'])

mean       4854.710955
median     4197.000000
std        3295.911843
min         813.750000
max       42621.000000
Name: deposit_cleaned, dtype: float64

In [ ]:
df.security_deposit.isnull().sum()

0

In [ ]:
tdf = pd.DataFrame({
    'val_1': [1, 2, 3, np.nan, np.nan],
    'val_2': [1, 2, 3, 4, np.nan],
})

In [ ]:
tdf.dropna()

,val_1,val_2
0,1.0,1.0
1,2.0,2.0
2,3.0,3.0


In [ ]:
tdf.dropna(how='all')

,val_1,val_2
0,1.0,1.0
1,2.0,2.0
2,3.0,3.0
3,NaN,4.0


In [ ]:
tdf

,val_1,val_2
0,1.0,1.0
1,2.0,2.0
2,3.0,3.0
3,NaN,4.0
4,NaN,NaN


In [ ]:
tdf['val_1'] = tdf.val_1.fillna(-99)

In [ ]:
tdf

,val_1,val_2
0,1.0,1.0
1,2.0,2.0
2,3.0,3.0
3,-99.0,4.0
4,-99.0,NaN


## Recap:

 **Useful methods for dealing with missing data**:
 - `pd.isna(VALUE)` and  `pd.isnull(VALUE)`: check if a value is `NaN`, returns `boolean`
 - `df.COLUMN_NAME.isna()` and `df.COLUMN_NAME.isnull()`: check each value in column to see if it is `NaN`, returns `boolean`
 - `df.dropna()`: will drop **every** row that has a missing value
     - optional arguments:
         - `axis`: drop the column
         - `how`: `any` or `all` drop if `any` value is missing, or `all` values in that row/col are missing
 - `df.fillna(VALUE_TO_FILL_WITH)`: will fill every missing value (`NaN`) with the value of your choice
 
**NOTE**: na and null do the **exact** same thing in Pandas, even the docs are the same. The duplicate methods are carried over from `R` which treated `na` and `null` as different values.

## Clean `property_condition`

In [ ]:
(df.isnull().sum() / len(df)).sort_values(ascending = False)

parking_space          0.882199
renovated_date         0.744764
efficiency_class       0.671466
energy_requirement     0.573298
cold_rent              0.442408
utilities              0.442408
heating_costs          0.442408
furnishing             0.417539
energy_certificate     0.409686
type_of_certificate    0.388743
energy_sources         0.353403
property_condition     0.344241
heating_type           0.325916
rent                   0.000000
region                 0.000000
condition              0.000000
rooms                  0.000000
n_missing              0.000000
year_construction      0.000000
space                  0.000000
security_deposit       0.000000
deposit_cleaned        0.000000
url                    0.000000
dtype: float64

In [ ]:
df.head()

,url,region,condition,rooms,rent,year_construction,space,cold_rent,utilities,heating_costs,security_deposit,parking_space,renovated_date,property_condition,furnishing,heating_type,energy_sources,energy_certificate,type_of_certificate,energy_requirement,efficiency_class,deposit_cleaned,n_missing
0,https://www.immobilienscout24.de/expose/116051687,Mitte,first_time_use,4.0,2659.00,2019,117.20,2.199€,+460€,in,3,120€,NaN,New,NaN,Underfloor Heating,NaN,NaN,Bedarfsausweis,NaN,B,7977.00,5
1,https://www.immobilienscout24.de/expose/115338103,Kreuzberg,first_time_use,1.0,1200.00,2020,29.33,"973,51€","+226,49€",in,2920.53,NaN,NaN,New,Gehobene Qualität,Central Heating,Fernwärme Energieausweis liegt vor,liegt vor,NaN,NaN,NaN,2920.53,5
2,https://www.immobilienscout24.de/expose/116458710,Köpenick,well_kept,2.0,979.00,1997,83.61,799€,+90€,+90€,3,NaN,NaN,Groomed,Normale Qualität,Central Heating,Fernwärme Energieausweis liegt vor,liegt vor,Verbrauchsausweis Energie­ver­brauchs­kennwert...,NaN,NaN,2937.00,4
3,https://www.immobilienscout24.de/expose/116573177,Wilmersdorf,well_kept,4.0,1830.22,1900,171.18,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5490.66,13
4,https://www.immobilienscout24.de/expose/115925878,Kreuzberg,first_time_use,2.0,2272.00,2020,88.27,NaN,NaN,NaN,nan,NaN,NaN,New,Gehobene Qualität,Central Heating,Fernwärme Energieausweis liegt zur Besichtigun...,liegt zur Besichtigung vor,NaN,NaN,NaN,6816.00,8


#### Categorical Features need to be handled differently than Continuous

Common methods:
 - Mode: May bias the dataset towards that category
 - Intuitive: Set the category based on domain knowledge, could be a problem if you are wrong
 - An `Unknown` Category: A catch all category for missing categories, especially useful if there is a reason the data is missing

In [ ]:
df.parking_space.isnull().sum() / len(df)

0.8821989528795812

In [ ]:
df.parking_space.mode()

0    120€
dtype: object

In [ ]:
df.property_condition.fillna('MISSING').value_counts()

MISSING                 263
New                     241
Mint Condition          103
Groomed                  65
Refurbished              29
After Renovation         26
Completely Renovated     21
Modernized               13
Needs Renovation          2
Well Maintained           1
Name: property_condition, dtype: int64

In [ ]:
df['property_condition'] = df.property_condition.fillna('Unknown')

In [ ]:
px.bar(df.property_condition.value_counts().reset_index(), x='index', y='property_condition')

## Clean `cold_rent`

In [ ]:
(df.isnull().sum() / len(df)).sort_values(ascending=False)

parking_space          0.882199
renovated_date         0.744764
efficiency_class       0.671466
energy_requirement     0.573298
cold_rent              0.442408
utilities              0.442408
heating_costs          0.442408
furnishing             0.417539
energy_certificate     0.409686
type_of_certificate    0.388743
energy_sources         0.353403
heating_type           0.325916
year_construction      0.000000
region                 0.000000
condition              0.000000
rooms                  0.000000
rent                   0.000000
n_missing              0.000000
space                  0.000000
security_deposit       0.000000
deposit_cleaned        0.000000
property_condition     0.000000
url                    0.000000
dtype: float64

**Since we have the warm rent for every sample, rather than use the median or mean, a better method might be to get the average warm/cold rent ratio and apply that to all missing values**

In [ ]:
df.cold_rent

0        2199
1      973.51
2         799
3         nan
4         nan
        ...  
759       nan
760       nan
761       nan
762       nan
763       nan
Name: cold_rent, Length: 764, dtype: object

In [ ]:
# reuse functions from before
df['cold_rent'] = df.cold_rent.apply(lambda x: remove_strings(x, ["€", "Euro"])).astype(str)
df['cold_rent'] = df.cold_rent.apply(convert_decimal)

In [ ]:
total_warm_and_cold = df[['rent', 'cold_rent']].dropna().astype(float).sum()
total_warm_and_cold

rent         1351180.56
cold_rent     672957.80
dtype: float64

In [ ]:
warm_cold_rent_ratio = total_warm_and_cold.cold_rent / total_warm_and_cold.rent
warm_cold_rent_ratio

0.4980517185652814

In [ ]:
px.histogram(df[['cold_rent']].astype(float).dropna(), x='cold_rent', marginal='box')

In [ ]:
df['cold_rent'] = df.cold_rent.astype(float)

In [ ]:
def get_cold_rent(row):
    if not pd.isna(row.cold_rent): 
        return float(row.cold_rent)
    else:
        return row.rent * warm_cold_rent_ratio

In [ ]:
df['cold_rent'] = df.apply(get_cold_rent, axis=1)

In [ ]:
px.histogram(df, x='cold_rent', marginal='box')

In [ ]:
df.to_pickle('berlin_housing_with_scraped_class_cleaned.pkl')